In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import ast
import json
from tqdm import tqdm_notebook
from sklearn.linear_model import LinearRegression
from sklearn import neighbors
import itertools

In [3]:
train=pd.read_csv('train.csv')
train.head()

,id,product_description,price
0,0,{'مشخصات فنی': 'متراژ: 150 سانتی متر\r\nپورت ی...,1080000
1,1,{'مشخصات فنی': 'CABLE LENTH:\r\n10feet \\/3M\r...,990000
2,2,{'مشخصات فنی': 'نوع کابل: KVM USB\r\nطول کابل:...,1680000
3,3,"{'دسته بندی': 'کابل کمبو', 'برند': 'متفرقه'}",4500000
4,4,{'مشخصات فنی': 'کیبورد: 6pin Mini-DIN Male - P...,700000


In [35]:
len(train)

68840

In [5]:
test=pd.read_csv("test.csv")
test.head()

,id,product_description
0,0,"{'سازگار با': 'C21N1401\r\nX455', 'ولتاژ باتری..."
1,1,"{'سازگار با': 'A31N1519\r\nX540', 'ولتاژ باتری..."
2,2,"{'تعداد فن': 'ندارد', 'نوع سیستم خنک کنندگی': ..."
3,3,{'سازگار با': 'A41N1424\r\nGL552\r\nZX50\r\nAS...
4,4,"{'سازگار با': 'AP31\r\nASUS 1008', 'ولتاژ باتر..."


In [34]:
len(test)

34262

In [6]:
list_train = []
for i in tqdm_notebook(range(len(train))):
    sample = train.values[i,1]
    sample = sample.replace('\\n','')
    sample = sample.replace('\\r','')
    sample = sample.replace('\\u200c','')
    sample = sample.replace('\\\\/','')
    list_train.append(ast.literal_eval(sample))

C:\Users\Admin\AppData\Local\Temp/ipykernel_7724/3877104310.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(train))):


  0%|          | 0/68840 [00:00<?, ?it/s]

In [7]:
list_test=[]
for i in tqdm_notebook(range(len(test))):
    sample = test.values[i,1]
    sample = sample.replace('\\n','')
    sample = sample.replace('\\r','')
    sample = sample.replace('\\u200c','')
    sample = sample.replace('\\\\/','')
    list_test.append(ast.literal_eval(sample))

C:\Users\Admin\AppData\Local\Temp/ipykernel_7724/3707715163.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(test))):


  0%|          | 0/34262 [00:00<?, ?it/s]

In [8]:
df_train = pd.DataFrame(list_train)  
df_train.rename({'دسته بندی': 'category'}, axis=1, inplace=True)
df_test = pd.DataFrame(list_test)  
df_test.rename({'دسته بندی': 'category'}, axis=1, inplace=True)

In [25]:
UniqueNames_train = df_train['category'].unique()
UniqueNames_test  = df_test['category'].unique()


array(['کابل کمبو', 'ریسوگراف', 'گام شمار', 'تجهیزات مدرن اداری', 'PBX',
       'کارت و ماژول', 'لوازم جانبی NAS و سرور', 'گیت وی', 'هاب سوئیچ',
       'رادیو تریگر', 'پرفراژ چک', 'کاغذ چاپ سریع', 'چتر فلاش آتلیه',
       'کپی برد', 'رول، درام و تونر فکس',
       'لوازم تعمیرات موبایل، تبلت و کالای دیجیتال', 'دیفیوزر',
       'کامپیوتر سرور', 'دستگاه کپی', 'فلاش آتلیه ای',
       'کتاب آموزش عکاسی و راهنمای دوربین', 'تجهیزات تلفن سانترال',
       'فلاش', 'کیبورد مخصوص تبلت', 'درام', 'لیزر', 'مبدلهای سازمانی',
       'آداپتور شبکه', 'پرینتر سه بعدی', 'پرس کارت و لمینت', 'پرزنتر',
       'اسکناس شمار', 'استند تبلت', 'برش دهندهی کاغذ', 'سر سه پایه',
       'توسعه دهنده محدوده بیسیم', 'کیوسک و استند لمسی', 'هدست تلفن',
       'تشخیص اصالت اسکناس', 'ذخیره ساز تحت شبکه (NAS)', 'سافت باکس',
       'مبدل فیبر', 'سایر لوازم جانبی پرینتر', 'آداپتور و کانورتر',
       'لنز دوربین', 'کاغذ خردکن', 'پچ پنل', 'رفلکتور', 'چادر عکاسی',
       'ریموت کنترل دوربین', 'کتابخوان و کاغذ دیجیتالی', 'اسکنر',
 

In [16]:
DataFrameDict_train = {elem : pd.DataFrame() for elem in UniqueNames_train}

for key in DataFrameDict_train.keys():
    DataFrameDict_train[key] = df_train[:][df_train.category== key]


In [17]:
DataFrameDict_test = {elem : pd.DataFrame() for elem in UniqueNames_test}

for key in DataFrameDict_test.keys():
    DataFrameDict_test[key] = df_test[:][df_test.category== key]

In [18]:
def to_numericcc(df): 
    dictt={}
    for col in df.columns:
        df[col] = df[col].apply(lambda x: str(x)) #اینجا تبدیل به رشته می‌کنم به خاطر این که یه سری ستون آرایه بود! اونا به مشکل بر میخوره برای مپ کردن
        u_list = list(df[col].unique()) # اینجا آیتم‌های یونیک رو میگیرم
        number_list = list(range(len(u_list))) #به تعداد اونا عدد جنریت میکنم که بعدش یک به یک قرار بدم
        r_map = {x:y for x,y in zip(u_list, number_list)} # حالا اون دیکشنری رو می‌سازم
        dictt.update(r_map)
        df[col] = df[col].replace(r_map) # اینجا هم رشته‌ها رو با اون عددی که مقابلش هست جا به جا میکنم
    return df ,dictt

In [54]:
ii={}
yt={}
kt={}
for i in UniqueNames_train:
    rr=DataFrameDict_train[i]
    cate=i
    rrr=rr.dropna(thresh=len(rr)/3, axis=1)
    rrr=rrr.drop('category', axis=1)
    index = rrr.index
    y=train['price'][index]
    #rrr=pd.get_dummies(rrr)
    X,eee=to_numericcc(rrr)
    yt.update({cate: eee})
    reg = LinearRegression().fit(X, y)
    ii[cate]=reg





In [46]:
def to_numeric_test(df,pp): 
    for col in df.columns:
        df[col] = df[col].apply(lambda x: str(x)) #اینجا تبدیل به رشته می‌کنم به خاطر این که یه سری ستون آرایه بود! اونا به مشکل بر میخوره برای مپ کردن
        u_list = list(df[col].unique()) # اینجا آیتم‌های یونیک رو میگیرمم
        
        df[col] = df[col].replace(pp) # اینجا هم رشته‌ها رو با اون عددی که مقابلش هست جا به جا میکنم
    return df

In [135]:
final={}

for i in UniqueNames_test:
    pp=yt[i]
    rr=DataFrameDict_test[i]
    cate=i
    rrr=rr.dropna(thresh=len(rr)/3, axis=1)
    rrr=rrr.drop('category', axis=1)
    X=to_numeric_test(rrr,pp)
    df1 = X.apply(pd.to_numeric, args=('coerce',))
    df2=df1.fillna(0)
    index = df2.index
    features=df2.columns
    for k in index:
        try:
            final[k]=float(ii[i].predict(df2.loc[[k]]))
        except:
            final[k]=1000.001
        

    
    





In [136]:
dict1 = OrderedDict(sorted(final.items()))


In [138]:
diccc={}
diccc['id']=dict1.keys()
diccc['price']=dict1.values()

In [141]:
final_final=pd.DataFrame.from_dict(diccc)

In [152]:
final_final.to_csv('output.csv',encoding='utf-8', index=False)

In [153]:
final_final.to_excel('output.xls',encoding='utf-8',index=False)

C:\Users\Admin\AppData\Local\Temp/ipykernel_7724/1264543221.py:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  final_final.to_excel('output.xls',encoding='utf-8',index=False)
